In [1]:
from google.colab import drive
mydrive = '/content/drive/'
drive.mount(mydrive)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import cv2, glob, os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from keras.preprocessing.image import img_to_array, load_img

import csv


In [0]:
def loading_data(path):
    print('Loading data...')
    imgs = []
    img_path = os.path.join(path , '*.jpg')
    imgs = imgs + [img_to_array(load_img(img)) for img in glob.glob(img_path)]
        # target_size remains same
          
    print('Total image number:' , len(imgs))
    print('Loading Done.')
    return imgs

In [0]:
def compute_avg(image, fore_indices):
    sum_ = 0
    for i in range(len(fore_indices[0])):
        sum_ = sum_ + image[ fore_indices[0][i] ][ fore_indices[1][i] ]
    avg = sum_ / len(fore_indices[0])
    return avg

# Use std to analysis

In [13]:
verbose = True
# src_path = '/content/drive/My Drive/Project/0425guava-classified'
src_path = '/content/drive/My Drive/Project/testing'
proc_path = '/content/drive/My Drive/Project/analysis_result'

dark = 50
max_ = 255

assert os.path.exists(src_path) is True, 'No file detected'
i = 1
for subpath in os.listdir(src_path):
    data_path = os.path.join(src_path, subpath)
    images = loading_data(data_path)

    SR = []
    Color = []
    for image in images:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ret,thres = cv2.threshold(image,dark,max_,cv2.THRESH_TOZERO)

        image_copy = thres
        back_indices = np.where(thres == 0)
        fore_indices = np.where(thres != 0)

        image_copy[back_indices] = compute_avg(image_copy, fore_indices)

        SR = SR + [np.std(image_copy)]
        Color = Color + [np.mean(image_copy)]
        if verbose:
            plt.subplot(1,2,1)
            plt.imshow(image, cmap = 'gray')
            plt.title('Origin')
            plt.subplot(1,2,2)
            plt.imshow(image_copy, cmap = 'gray')
            plt.title('After calibration')
            plt.show()
            print('mean(after calibration) = ', np.mean(image))
            print('std(after calibration) = ', np.std(image))
            print('=================================================')
            print('mean(after calibration) = ', np.mean(image_copy))
            print('std(after calibration) = ', np.std(image_copy))
            print('=================================================')
            xx, yy = np.mgrid[0:image.shape[0], 0:image.shape[1]]
            fig = plt.figure(figsize=(12,12))
            ax = fig.gca(projection='3d')
            ax.plot_surface(xx, yy, image ,rstride=1, cstride=1, cmap='gray', linewidth=0)
            ax.set_zlim3d(0,500)
            plt.show()
            #########
            xx, yy = np.mgrid[0:image_copy.shape[0], 0:image_copy.shape[1]]
            fig = plt.figure(figsize=(12,12))
            ax = fig.gca(projection='3d')
            ax.plot_surface(xx, yy, image_copy ,rstride=1, cstride=1, cmap='gray', linewidth=0)
            ax.set_zlim3d(0,500)
            plt.title('Surface graph')
            plt.show()

    image_names = os.listdir(data_path)
    directory = os.path.join(proc_path, subpath)        
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(os.path.join(directory,'output'+str(i)+'.csv'), 'w', newline='') as csvfile:
        # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)

        # 寫入一列資料
        writer.writerow(image_names)

        # 寫入另外幾列資料
        writer.writerow(SR)
        writer.writerow(Color)
    i = i + 1
"""
    print('Saving...')
    image_names = os.listdir(data_path)
    directory = os.path.join(proc_path, subpath)
    if not os.path.exists(directory):
        os.makedirs(directory)
    kmeans_image_save(directory, image_names, segmented_images)
"""
print('Done.')



Output hidden; open in https://colab.research.google.com to view.

Load .csv

In [0]:
for subpath in os.listdir(src_path):
    with open('/etc/passwd', newline='') as csvfile:
